## Задание 1
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.


In [12]:
import pandas as pd

In [2]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [3]:
data = pd.read_csv('keywords.csv')

In [4]:
def geo_identife(row):
    for key,value in geo_data.items():
        for i in value:
            if i in row['keyword']:
                return key
    else:
        return 'undefined'
    

In [5]:
data['region']  = data.apply(geo_identife, axis = 1)   

In [6]:
data['region'].value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

## Задание 2
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
	- оценка 2 и меньше - низкий рейтинг
	- оценка 4 и меньше - средний рейтинг
	- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [25]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
def classification(row):
    if (int(row['rating']) <= 2):
        return 'низкий рейтинг'
    elif (int(row['rating']) <= 4):
        return 'средний рейтиг'
    else:
        return 'высокий рейтинг'

In [9]:
ratings['class'] =  ratings.apply(classification, axis=1)

## Задание 3
Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.


In [10]:
# считаем скоько оценок проставил каждый пользователь
ratings_fitered = ratings.groupby('userId').count()[['movieId']] 

In [11]:
# оставляем пользователей, которые поставили больше 100 оценок 
ratings_fitered = ratings_fitered[ratings_fitered['movieId'] > 100].reset_index()

In [12]:
# делаем список этих поьзователей
list_userId = ratings_fitered['userId'].tolist()

In [13]:
# фильтруем таблицу по этим пользователям
ratings = ratings[ratings['userId'].isin(list_userId)]

In [32]:
# находим min и max timestamp для каждого пользователя 
ratings_grouped = ratings.groupby('userId').agg(['max','min'])['timestamp'].reset_index()

In [41]:
# считает число дней между min и max timestamp для каждого пользователя 
def lifetime(row):
    from datetime import datetime
    date_max = datetime.fromtimestamp(int(row['max']))
    date_min = datetime.fromtimestamp(int(row['min']))
    return (date_max -  date_min).days

In [35]:
ratings_grouped['lifetime'] = ratings_grouped.apply(lifetime, axis = 1)

In [36]:
ratings_grouped

,userId,max,min,lifetime
0,1,965719662,964980499,8
1,4,1007574542,945078428,723
2,6,845556915,845553109,0
3,7,1176181731,1106635416,804
4,10,1455619275,1455301553,3
5,15,1510578158,1299424762,2443
6,17,1326943248,1305696226,245
7,18,1534786448,1455049328,922
8,19,965712882,965701107,0
9,20,1054147519,1054036159,1


In [40]:
# находим среднее значеие lifetime
int(ratings_grouped['lifetime'].mean())

517

## Задание 4
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
	- для каждой строки пройдите по всем годам списка years
	- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
	- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга


In [1]:
years = [i for i in range(1950,2011)]

In [3]:
import pandas as pd

In [8]:
def production_year(row):
    for year in years:
        if str(year) in row['title']:
            return year
    else:
        return '1900'

In [6]:
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
movies['year'] = movies.apply(production_year,axis=1)

In [45]:
movies.drop_duplicates(subset = 'movieId', keep = 'first', inplace = True)
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [47]:
ratings.drop_duplicates(subset = 'movieId', keep = 'first', inplace = True)

In [49]:
joined = movies.merge(ratings, how ='left', on = 'movieId')

In [51]:
len(movies) == len(joined)

True

In [52]:
joined.head()

,movieId,title,genres,year,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,1.0,4.0,964982703.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,6.0,4.0,845553522.0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,1.0,4.0,964981247.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,6.0,3.0,845554349.0
4,5,Father of the Bride Part II (1995),Comedy,1995,6.0,5.0,845553938.0


In [57]:
joined.groupby('year').mean()[['rating']].reset_index().sort_values('rating', ascending=False)

,year,rating
21,1971,3.925532
18,1968,3.845238
1,1951,3.833333
16,1966,3.750000
7,1957,3.742424
5,1955,3.694444
11,1961,3.681818
24,1974,3.681818
26,1976,3.670455
22,1972,3.650000
